#Function to Record Audio in Colab

In [56]:
# all imports
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  with open('audio.mp3','wb') as f:
    f.write(b)
  return 'audio.mp3'  # or webm ?



#Record Audio

Pass the number of seconds as a parameter to the function 

In [67]:
record(10)

<IPython.core.display.Javascript object>

'audio.mp3'

#Transcribe

In [68]:
import requests
url = "https://whisper.lablab.ai/asr"
payload={}
files=[
('audio_file',('audio.mp3',open('audio.mp3','rb'),'audio/mpeg'))
]
response = requests.request("POST", url, data=payload, files=files)

# The response is a non-scriptable class so we have to import json to view the content
import json
content = json.loads(response.content)
print(content['text'])

 My name is Mohammad Abuzar and I am a student of software engineering.
